# Assignment 2 - CS661

**Group No.: 76**  
**Krishna Kumar Bais (241110038)**  
**Milan Roy (241110042)**  

## 3D Isosurface Visualization with Histogram Analysis

### Instructions
1. Ensure that VTK, NumPy, Plotly, ipywidgets, ipykernel and nbformat are installed before running the script.  
2. The input data file `mixture.vti` should be present in the same directory as the script to ensure successful data loading.  
3. The script is designed to run successfully in **VS Code**. It may not function properly in Jupyter Notebook or Google Colab due to compatibility issues.  
4. The script reads volumetric data from `mixture.vti`, extracts point data, and visualizes it using Plotly's Isosurface plot.  
5. An interactive slider allows the user to adjust the isovalue, dynamically updating the visualization.  
6. The histogram adjacent to the isosurface visualization shows the distribution of scalar values filtered within a range around the selected isovalue.  

### Execution Instructions
1. Open the script in **VS Code**.  
2. Ensure that the data file `mixture.vti` is in the same directory as the script.  
3. Run the script in VS Code.  
4. Use the provided slider to change the isovalue and observe the updates in the isosurface visualization and histogram.  
5. Click the 'Reset' button to restore the initial settings.  

### Libraries Required
- VTK  
- NumPy  
- Plotly  
- ipywidgets  
- ipykernel
- nbformat

### Sample Installation Commands
```bash
pip install vtk numpy plotly ipywidgets ipykernel notebook nbformat
```
**Please restart VSCode after installing the above libraries.**

# Code

In [ ]:
!pip install vtk numpy plotly ipywidgets ipykernel nbformat

In [ ]:
import vtk
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Data Reading and Extraction
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName('mixture.vti')
reader.Update()
data = reader.GetOutput()

num_points = data.GetNumberOfPoints()
pdata = data.GetPointData()
image_data = pdata.GetArray('ImageFile')

# Extracting coordinate points and values
x, y, z, values = [], [], [], []
for i in range(num_points):
    point = data.GetPoint(i)
    x.append(point[0])
    y.append(point[1])
    z.append(point[2])
    values.append(image_data.GetValue(i))

# Converting the values to numpy arrays
x = np.array(x)
y = np.array(y)
z = np.array(z)
values = np.array(values)

# Declare the has_reset flag to control reset behaviour
has_reset = True

# Function to Update Plots
def update_visuals(isovalue):
    global has_reset
    
    # Filtering values for histogram
    if has_reset:
        has_reset = False
        filtered_values = values.copy()
    else:
        filtered_values = values[(values >= (isovalue-0.25)) & (values <= (isovalue+0.25))]

    # Creating the Isosurface
    isosurface = go.Figure(data=go.Isosurface(
        x=x,
        y=y,
        z=z,
        value=values,
        isomin=isovalue,
        isomax=isovalue,
        surface_count=1,
        showscale=False,
        colorscale='plasma',
        cmin=min(values),
        cmax=max(values)
    ))

    # Creating the histogram
    # The nbinsx value was determined by testing different options (10, 20, 30) to find the one that 
    # best matches the histogram in the PDF.
    histogram = go.Figure(data=[go.Histogram(x=filtered_values, nbinsx=30)])

    # Creating Subplots
    fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'scene'}, {'type': 'histogram'}]], horizontal_spacing=0.1)
    fig.add_trace(isosurface.data[0], row=1, col=1)
    fig.add_trace(histogram.data[0], row=1, col=2)

    # Updating the layout to remove axis ticks and add axis labels
    fig.update_layout(
        width = 950,
        scene1=dict(
            xaxis=dict(showticklabels=False),
            yaxis=dict(showticklabels=False),
            zaxis=dict(showticklabels=False)
        ),
        xaxis1=dict(title="Vortex Scalar Values",),
        yaxis1=dict(title="Frequency"),

    )

    fig.show()

# Function for reset
def reset_plots(_):
    # Set the has_reset flag and reset the slider value.
    global has_reset
    has_reset = True
    slider.value = 0.0
    # Changing Slider value automaticaly calls update_visuals()

# Slider
slider = widgets.FloatSlider(value=0.0,
                             min=min(values), 
                             max=max(values), 
                             step=0.01, 
                             description='Isovalue:',
                             continuous_update=False,
                            readout=True, 
                            readout_format='.2f')

# Reset Button
reset_button = widgets.Button(description="Reset", button_style='primary')
reset_button.on_click(reset_plots)

interactive_plot = widgets.interactive_output(update_visuals, {'isovalue':slider})

# Placing slider and reset button side by side using a horizontal box
controls = widgets.HBox([slider, reset_button])

# Placing the controls and interactive plot below the other using a vertical box.
# Also adding a border.
container = widgets.VBox([controls, interactive_plot],
    layout=widgets.Layout(
        border='3px solid blue',  
        padding='15px',
        margin='20px',
        width='98%'  
    )
)

# Display the container
display(container)
